# Imports

In [4]:
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))

import pandas as pd
import duckdb

In [5]:
#print(sys.path)

____

___

# Read From duckbd

In [6]:
con = duckdb.connect("data/candles.duckdb")
#print(con.execute("SELECT * FROM candles ORDER BY timestamp DESC LIMIT 5").fetchall())
query = """SELECT * 
           FROM candles
        """
query_results = con.execute(query).fetchall()

In [7]:
print(query_results)

[(datetime.datetime(2025, 5, 12, 10, 36, 55, 214398), 1747040400.0, 104460.0, 104507.4, 104117.0, 104316.2531), (datetime.datetime(2025, 5, 12, 10, 37, 14, 475778), 1747040400.0, 104460.0, 104507.4, 104117.0, 104316.25505), (datetime.datetime(2025, 5, 12, 10, 37, 16, 6897), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.21024), (datetime.datetime(2025, 5, 12, 10, 37, 19, 56725), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.893), (datetime.datetime(2025, 5, 12, 10, 37, 28, 111583), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89343), (datetime.datetime(2025, 5, 12, 10, 37, 32, 821989), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89613), (datetime.datetime(2025, 5, 12, 10, 37, 42, 10585), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89641), (datetime.datetime(2025, 5, 12, 10, 37, 51, 671914), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.8979), (datetime.datetime(2025, 5, 12, 10, 37, 54, 956892), 1747040400.0, 104460.0, 104507.4, 104117.0, 104317.89804),

In [8]:
con.close()

___

# Pipe the results to a Pandas Dataframe

In [9]:
df = pd.DataFrame(query_results)

In [10]:
df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']

In [11]:
df.head()

,Datetime,Open,High,Low,Close,Volume
0,2025-05-12 10:36:55.214398,1.747040e+09,104460.0,104507.4,104117.0,104316.25310
1,2025-05-12 10:37:14.475778,1.747040e+09,104460.0,104507.4,104117.0,104316.25505
2,2025-05-12 10:37:16.006897,1.747040e+09,104460.0,104507.4,104117.0,104317.21024
3,2025-05-12 10:37:19.056725,1.747040e+09,104460.0,104507.4,104117.0,104317.89300
4,2025-05-12 10:37:28.111583,1.747040e+09,104460.0,104507.4,104117.0,104317.89343


In [12]:
len(df)

1172

____


____

In [13]:
# strategy/ta.py
import sys
import os
#sys.path.append(os.path.dirname(os.path.abspath(__file__)))
import sys
sys.path.append(r'C:\Users\trrallele\Momentum Metropolitan\REALEARN\MVP Scaffold\mvpenv\Lib\site-packages')


import duckdb
import pandas as pd
import pandas_ta as ta
from datetime import datetime, timedelta

DB_PATH = "data/candles.duckdb"

def load_recent_candles(hours=200):
    con = duckdb.connect(DB_PATH)
    query = f"""
    SELECT * FROM candles
    WHERE timestamp >= NOW() - INTERVAL {hours} HOUR
    ORDER BY timestamp ASC
    """
    df = con.execute(query).fetchdf()
    con.close()
    return df

def compute_indicators(df):
    df.set_index('timestamp', inplace=True)

    # Add 3 basic indicators
    df['rsi'] = ta.rsi(df['close'], length=14)
    df['ema_50'] = ta.ema(df['close'], length=50)
    df['ema_200'] = ta.ema(df['close'], length=200)

    return df

def generate_signal(df):
    latest = df.iloc[-1]

    signal = "hold"
    reason = ""

    # Strategy logic
    if latest['ema_50'] > latest['ema_200'] and latest['rsi'] < 30:
        signal = "buy"
        reason = "Golden cross + RSI oversold"
    elif latest['ema_50'] < latest['ema_200'] and latest['rsi'] > 70:
        signal = "sell"
        reason = "Death cross + RSI overbought"

    return signal, reason

def get_ta_signal():
    df = load_recent_candles()
    df = compute_indicators(df)
    signal, reason = generate_signal(df)
    print(f"TA Signal: {signal.upper()} | Reason: {reason}")
    return signal, reason


In [14]:
get_ta_signal()

TA Signal: HOLD | Reason: 


('hold', '')